# Prevendo as notas de matemática do ENEM 2016 
### Nesse desafio, devemos elaborar um modelo de regressão que seja capaz de prever a nota de matemática do ENEM 2016

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression ,Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

In [2]:
df_treino = pd.read_csv('train.csv', index_col='Unnamed: 0')
df_teste = pd.read_csv('test.csv')

# Visão Geral

Aqui vamos obter a visão geral dos dataset que estamos trabalhando, obtendo informações comoo número de observações e colunas, o nomes das variáveis, o tipo de dados presentes, a quantidade de dados ausentes, etc.

In [3]:
df_treino.shape

(13730, 166)

In [4]:
df_teste.shape

(4576, 47)

In [5]:
df_treino.columns

Index(['NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA',
       'NO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA',
       'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
       ...
       'Q041', 'Q042', 'Q043', 'Q044', 'Q045', 'Q046', 'Q047', 'Q048', 'Q049',
       'Q050'],
      dtype='object', length=166)

In [6]:
df_teste.columns

Index(['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'NU_IDADE',
       'TP_SEXO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
       'TP_DEPENDENCIA_ADM_ESC', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ',
       'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_SABATISTA', 'IN_GESTANTE',
       'IN_IDOSO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT',
       'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'TP_LINGUA',
       'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
       'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002',
       'Q006', 'Q024', 'Q025', 'Q026', 'Q027', 'Q047'],
      dtype='object')

In [7]:
df_treino.dtypes.value_counts()

int64      78
object     60
float64    28
dtype: int64

In [8]:
df_teste.dtypes.value_counts()

int64      20
object     15
float64    12
dtype: int64

In [9]:
# Dados faltantes no dataset de treino
df_treino.isna().sum()[df_treino.isna().sum()>0].sort_values(ascending=False)

NO_ENTIDADE_CERTIFICACAO       12092
CO_UF_ENTIDADE_CERTIFICACAO    12092
SG_UF_ENTIDADE_CERTIFICACAO    12092
Q041                           10792
SG_UF_ESC                       9448
TP_LOCALIZACAO_ESC              9448
TP_SIT_FUNC_ESC                 9448
CO_UF_ESC                       9448
NO_MUNICIPIO_ESC                9448
CO_MUNICIPIO_ESC                9448
CO_ESCOLA                       9448
TP_ENSINO                       9448
TP_DEPENDENCIA_ADM_ESC          9448
Q032                            7376
Q031                            7376
Q028                            7376
Q033                            7376
Q030                            7375
Q029                            7375
Q027                            7373
NU_NOTA_COMP1                   3597
NU_NOTA_LC                      3597
NU_NOTA_REDACAO                 3597
NU_NOTA_MT                      3597
TP_STATUS_REDACAO               3597
NU_NOTA_COMP5                   3597
TX_RESPOSTAS_LC                 3597
T

In [10]:
# Dados faltantes no dataset de teste
df_teste.isna().sum()[df_teste.isna().sum()>0].sort_values(ascending=False)

TP_DEPENDENCIA_ADM_ESC    3096
TP_ENSINO                 3096
Q027                      2488
NU_NOTA_REDACAO           1199
NU_NOTA_COMP5             1199
NU_NOTA_COMP4             1199
NU_NOTA_COMP3             1199
NU_NOTA_COMP2             1199
NU_NOTA_COMP1             1199
TP_STATUS_REDACAO         1199
NU_NOTA_LC                1199
NU_NOTA_CH                1134
NU_NOTA_CN                1134
dtype: int64

### Pré-processamento dos dados

Primeiramente, iremos filtrar o dataset de treinamento pelas features que o dataset de teste possui, reduzindo assim a quantidade de variáveis a serem tratadas

In [11]:
colunas = list(df_teste.columns)
# Lembrando de deixar a coluna 'NU_NOTA_MT'
colunas.append('NU_NOTA_MT')
df_treino = df_treino[colunas]

Assim, partimos para a analisar das features. Primeiro, levando em consideração os dados faltantes(NAs) em termos absoluto e percental.

In [12]:
df_treino.isna().sum()[df_treino.isna().sum()>0].sort_values(ascending=False)

TP_DEPENDENCIA_ADM_ESC    9448
TP_ENSINO                 9448
Q027                      7373
NU_NOTA_MT                3597
NU_NOTA_REDACAO           3597
NU_NOTA_COMP5             3597
NU_NOTA_COMP4             3597
NU_NOTA_COMP3             3597
NU_NOTA_COMP2             3597
NU_NOTA_COMP1             3597
TP_STATUS_REDACAO         3597
NU_NOTA_LC                3597
NU_NOTA_CH                3389
NU_NOTA_CN                3389
dtype: int64

In [13]:
df_treino.isna().sum()[df_treino.isna().sum()>0].sort_values(ascending=False)/df_treino.shape[0]

TP_DEPENDENCIA_ADM_ESC    0.688128
TP_ENSINO                 0.688128
Q027                      0.536999
NU_NOTA_MT                0.261981
NU_NOTA_REDACAO           0.261981
NU_NOTA_COMP5             0.261981
NU_NOTA_COMP4             0.261981
NU_NOTA_COMP3             0.261981
NU_NOTA_COMP2             0.261981
NU_NOTA_COMP1             0.261981
TP_STATUS_REDACAO         0.261981
NU_NOTA_LC                0.261981
NU_NOTA_CH                0.246832
NU_NOTA_CN                0.246832
dtype: float64

Como as variáveis `TP_DEPENDENCIA_ADM_ESC` e `TP_ENSINO` possuem mais de 2/3 de valores faltantes, decidimos excluí-las

In [14]:
df_treino = df_treino.drop(['TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO'], axis=1)

Em seguida, imputamos a nota 0 para aqueles estudantes que faltaram à prova (`TP_PRESENCA_XX` igual a 0) ou que foram eliminados (`TP_PRESENCA_XX` igual a 2). Como não há a coluna de presença na prova de matemática em ambos datasetes, utilizando a coluna de presença na prova de linguagens e códigos, uma vez que elas são realizadas juntas. 

In [15]:
df_treino.NU_NOTA_CN.loc[df_treino.TP_PRESENCA_CN == 0] = 0
df_treino.NU_NOTA_CN.loc[df_treino.TP_PRESENCA_CN == 2] = 0

df_treino.NU_NOTA_CH.loc[df_treino.TP_PRESENCA_CH == 0] = 0
df_treino.NU_NOTA_CH.loc[df_treino.TP_PRESENCA_CH == 2] = 0

df_treino.NU_NOTA_LC.loc[df_treino.TP_PRESENCA_LC == 0] = 0
df_treino.NU_NOTA_LC.loc[df_treino.TP_PRESENCA_LC == 2] = 0

df_treino.NU_NOTA_MT.loc[df_treino.TP_PRESENCA_LC== 0] = 0
df_treino.NU_NOTA_MT.loc[df_treino.TP_PRESENCA_LC == 2] = 0

c:\users\dell\codenation\venv\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


De maneira semelhante, os valores faltantes para as compentências também foram imputadas com 0 seguindo a nota de redação ao passo que o status da redação foi alterado para a categoria 4 (prova em branco).

In [16]:
df_treino.NU_NOTA_COMP1.loc[pd.isnull(df_treino.NU_NOTA_COMP1)] = 0
df_treino.NU_NOTA_COMP2.loc[pd.isnull(df_treino.NU_NOTA_COMP2)] = 0
df_treino.NU_NOTA_COMP3.loc[pd.isnull(df_treino.NU_NOTA_COMP3)] = 0
df_treino.NU_NOTA_COMP4.loc[pd.isnull(df_treino.NU_NOTA_COMP4)] = 0
df_treino.NU_NOTA_COMP5.loc[pd.isnull(df_treino.NU_NOTA_COMP5)] = 0
df_treino.NU_NOTA_REDACAO.loc[pd.isnull(df_treino.NU_NOTA_REDACAO)] = 0
df_treino.TP_STATUS_REDACAO.loc[pd.isnull(df_treino.TP_STATUS_REDACAO)] = 4

Alteramos o Status da redação para reduzir suas categorias para apenas duas: 1 para redação sem problemas e 0 para redação com problemas(redação anulada, cópia do texto motivador, em branco, fere os direitos humanos, fuga ao tema, não atendimento ao tipo, texto insuficiente ou parte desconectada)

In [17]:
df_treino.TP_STATUS_REDACAO = df_treino.TP_STATUS_REDACAO.where(cond=df_treino.TP_STATUS_REDACAO==1, other=0)

A última variável com dados faltantes, a`Q027` não apresentava uma opção para aqueles que não havima começado uma atividade remunada, então optamos por criar uma, a 'S' 

In [18]:
df_treino.Q027.loc[df_treino.Q027.isna()==True] = 'S'

Em seguida, excluimos aquelas variáveis que não agregam à análise

In [19]:
df_treino = df_treino.drop(['SG_UF_RESIDENCIA','NU_INSCRICAO','CO_PROVA_CN','CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT'], axis = 1)

Por fim, faremos o enconding das variáveis categóricas com o intuito de transformá-las em variáveis contínuas para que sejam compatíveis com o modelo de regressão a ser aplicado.

In [20]:
labelenconder = LabelEncoder()

In [21]:
df_treino.TP_SEXO = labelenconder.fit_transform(df_treino.TP_SEXO)
df_treino.Q001 = labelenconder.fit_transform(df_treino.Q001)
df_treino.Q002 = labelenconder.fit_transform(df_treino.Q002)
df_treino.Q006 = labelenconder.fit_transform(df_treino.Q006)
df_treino.Q024 = labelenconder.fit_transform(df_treino.Q024)
df_treino.Q025 = labelenconder.fit_transform(df_treino.Q025)
df_treino.Q026 = labelenconder.fit_transform(df_treino.Q026)
df_treino.Q027 = labelenconder.fit_transform(df_treino.Q027)
df_treino.Q047 = labelenconder.fit_transform(df_treino.Q047)

In [22]:
df_treino = df_treino.drop(['TP_COR_RACA','IN_CEGUEIRA','IN_SURDEZ','IN_DISLEXIA', 'IN_DISCALCULIA','IN_SABATISTA',
         'IN_GESTANTE','IN_IDOSO', 'TP_NACIONALIDADE','TP_ST_CONCLUSAO','IN_BAIXA_VISAO',
         'CO_UF_RESIDENCIA','TP_SEXO','IN_TREINEIRO','TP_ANO_CONCLUIU', 'TP_LINGUA','Q001','Q002','Q024','Q025','Q027','Q047'],
        axis = 1)

# Criação de uma função pra tratamento dos dados de teste

In [23]:
def tratamento(df_teste):
    df_teste= df_teste.drop(['TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO'], axis=1)
    df_teste.NU_NOTA_CN.loc[df_teste.TP_PRESENCA_CN == 0] = 0
    df_teste.NU_NOTA_CN.loc[df_teste.TP_PRESENCA_CN == 2] = 0
    df_teste.NU_NOTA_CH.loc[df_teste.TP_PRESENCA_CH == 0] = 0
    df_teste.NU_NOTA_CH.loc[df_teste.TP_PRESENCA_CH == 2] = 0
    df_teste.NU_NOTA_LC.loc[df_teste.TP_PRESENCA_LC == 0] = 0
    df_teste.NU_NOTA_LC.loc[df_teste.TP_PRESENCA_LC == 2] = 0
    df_teste.NU_NOTA_COMP1.loc[pd.isnull(df_teste.NU_NOTA_COMP1)] = 0
    df_teste.NU_NOTA_COMP2.loc[pd.isnull(df_teste.NU_NOTA_COMP2)] = 0
    df_teste.NU_NOTA_COMP3.loc[pd.isnull(df_teste.NU_NOTA_COMP3)] = 0
    df_teste.NU_NOTA_COMP4.loc[pd.isnull(df_teste.NU_NOTA_COMP4)] = 0
    df_teste.NU_NOTA_COMP5.loc[pd.isnull(df_teste.NU_NOTA_COMP5)] = 0
    df_teste.NU_NOTA_REDACAO.loc[pd.isnull(df_teste.NU_NOTA_REDACAO)] = 0
    df_teste.TP_STATUS_REDACAO.loc[pd.isnull(df_teste.TP_STATUS_REDACAO)] = 4
    df_teste.TP_STATUS_REDACAO = df_teste.TP_STATUS_REDACAO.where(cond=df_teste.TP_STATUS_REDACAO==1, other=0)
    df_teste.Q027.loc[df_teste.Q027.isna()==True] = 'S'
    df_teste.drop(['SG_UF_RESIDENCIA','NU_INSCRICAO','CO_PROVA_CN','CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT'], axis = 1, inplace = True)
    df_teste.TP_SEXO = labelenconder.fit_transform(df_teste.TP_SEXO)
    df_teste.Q001 = labelenconder.fit_transform(df_teste.Q001)
    df_teste.Q002 = labelenconder.fit_transform(df_teste.Q002)
    df_teste.Q006 = labelenconder.fit_transform(df_teste.Q006)
    df_teste.Q024 = labelenconder.fit_transform(df_teste.Q024)
    df_teste.Q025 = labelenconder.fit_transform(df_teste.Q025)
    df_teste.Q026 = labelenconder.fit_transform(df_teste.Q026)
    df_teste.Q027 = labelenconder.fit_transform(df_teste.Q027)
    df_teste.Q047 = labelenconder.fit_transform(df_teste.Q047)
    return df_teste

In [24]:
teste = tratamento(df_teste)

In [25]:
teste = teste.drop(['TP_COR_RACA','IN_CEGUEIRA','IN_SURDEZ','IN_DISLEXIA', 'IN_DISCALCULIA','IN_SABATISTA',
         'IN_GESTANTE','IN_IDOSO', 'TP_NACIONALIDADE','TP_ST_CONCLUSAO','IN_BAIXA_VISAO',
         'CO_UF_RESIDENCIA','TP_SEXO','IN_TREINEIRO','TP_ANO_CONCLUIU', 'TP_LINGUA','Q001','Q002','Q024','Q025','Q027','Q047'],
        axis = 1)

# Prepação para o modelo de regressão

A prepação para modelo de regressão envolve a utilização do dataset de treinamento por meio do Train-Test Split, em que lançaremos mão do método do próprio Sklearn

In [26]:
scaler = StandardScaler()

In [27]:
x = df_treino.drop('NU_NOTA_MT', axis=1)
x = scaler.fit_transform(x)
y = df_treino.NU_NOTA_MT


In [28]:
X_treino, X_teste, Y_treino, Y_teste = train_test_split(x, y, test_size = 0.3332847778587036)

In [29]:
print(X_treino.shape, X_teste.shape, Y_treino.shape, Y_teste.shape)

(9154, 17) (4576, 17) (9154,) (4576,)


# Treinando o modelo de regressão

In [30]:
linear = LinearRegression()

In [31]:
linear.fit(X_treino, Y_treino)

LinearRegression()

In [32]:
y_pred = linear.predict(X_teste)

In [33]:
y_pred

array([501.57669636,  30.88919636, 501.92044636, ..., 486.54544636,
       559.67044636, 393.01419636])

In [34]:
mean_squared_error(Y_teste, y_pred)

4601.536224297282

In [35]:
linear.score(X_treino, Y_treino)

0.9127076261537909

In [36]:
linear.score(X_teste, Y_teste)

0.9119612961958233

# Aplicação do modelo ao dataset de teste

In [37]:
set_test = scaler.fit_transform(teste)

In [38]:
pred_final = linear.predict(set_test)

In [39]:
set_test.shape

(4576, 17)

In [40]:
pd.Series(pred_final)

0      -5.377472e+08
1       5.752635e+10
2      -1.895220e+10
3      -1.197119e+11
4      -1.614203e+10
5       5.142256e+10
6       5.906482e+10
7       1.013138e+10
8       1.155905e+11
9       5.752635e+10
10     -1.197119e+11
11     -1.197119e+11
12      6.328337e+10
13      9.309517e+09
14      9.384038e+10
15      6.791374e+10
16     -1.197119e+11
17     -1.197119e+11
18     -1.197119e+11
19     -3.587667e+09
20      6.027148e+10
21     -1.197119e+11
22     -1.197119e+11
23      8.427486e+10
24      1.247442e+11
25      4.346053e+10
26      1.232707e+11
27      1.630021e+10
28      1.774663e+10
29     -1.197119e+11
            ...     
4546   -1.197119e+11
4547   -1.197119e+11
4548    4.685722e+10
4549   -1.197119e+11
4550    2.995423e+10
4551   -3.869405e+09
4552    3.921496e+10
4553    6.702684e+10
4554    3.461160e+10
4555    7.055581e+10
4556    5.410265e+10
4557    6.363015e+10
4558    4.947228e+10
4559    1.205465e+10
4560    1.186404e+11
4561    1.198961e+10
4562    3.153

In [41]:
resposta = pd.DataFrame({'NU_INSCRICAO':df_teste.NU_INSCRICAO, 'NU_NOTA_MT':pred_final})

In [42]:
resposta.to_csv('answer.csv',index=False)